# BPR Bayesian Personalized Ranking

---

ThetaLog.com

In [1]:
# Load các thư viện cần thiết
import os
import math
import numpy as np
import pandas as pd
from urllib.request import urlopen
from zipfile import ZipFile
from scipy.sparse import csr_matrix, dok_matrix
from sklearn.metrics import roc_auc_score

np.random.seed(12)

In [2]:
def load_movielens_data():
    """
    Tải tập tin dữ liệu từ trang chủ grouplens
    Tập dữ liệu: ml-100k

    :return dataframe: pandas df
    """
    zipurl = 'http://files.grouplens.org/datasets/movielens/ml-100k.zip'
    zipresp = urlopen(zipurl)

    # Nếu không có thì tài về từ mạng internet
    if not os.path.isdir('ml-100k'):
        print('Downloading: http://files.grouplens.org/datasets/movielens/ml-100k.zip')
        with open("ml-100k.zip", "wb") as file:
            file.write(zipresp.read())
            file.close()
        print('Completed!')

    # Giải nén
    zipdata = ZipFile("ml-100k.zip")
    zipdata.extractall(path = '')
    zipdata.close()

    # Load dữ liệu từ Pandas
    file_path = os.path.join('ml-100k', 'u.data')
    names = ['users', 'items', 'ratings', 'timestamp']
    dataframe = pd.read_csv(file_path, sep = '\t', names = names)
    return dataframe

In [3]:
dataframe = load_movielens_data()

In [4]:
# Xem thử dữ liệu trông thế nào
dataframe.head()

,users,items,ratings,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
def convert_to_bpr_mat(dataframe, threshold=3):
    """
    Chuyển đổi DataFrame MovieLens 100K ban đầu sang ma trận BPR
    Mỗi dòng là Users
    Mỗi cột là Item
    Định dạng ma trận thưa

    :param dataframe: pandas df movielens 100K
    :param threshold: ngưỡng ratings chấp nhận nên khuyến nghị
    :return bpr_mat: np.array - ma trận thưa bpr
    """
    tempdf = dataframe.copy()
    tempdf['positive'] = tempdf['ratings'].apply(func=lambda x: 0 if x < threshold else 1)

    # Vì tập dữ liệu này đánh index từ 1 nên chuyển sang kiểu category
    # để tránh việc chúng ta có ma trận
    tempdf['users'] = tempdf['users'].astype('category')
    tempdf['items'] = tempdf['items'].astype('category')

    bpr_mat = csr_matrix((tempdf['positive'],
                          (tempdf['users'].cat.codes,
                           tempdf['items'].cat.codes)))
    bpr_mat.eliminate_zeros()
    del tempdf
    return bpr_mat

In [6]:
bpr_mat = convert_to_bpr_mat(dataframe)

In [7]:
def split_to_train_test(bpr_mat, test_ratio = 0.2, verbose=True):
    """
    Chia tập dữ liệu ra thành tập train & tập test

    :param bpr_mat: ma trận bpr
    :param test_ratio: float - tỉ lệ test set

    :return train: ma trận bpr train
    :return test: ma trận bpr test
    """
    # Số lượng người dùng
    n_users = bpr_mat.shape[0]
    # Dùng ma trận thưa Dictionary Of Keys tối ưu hơn cho công đoạn này
    train = bpr_mat.copy().todok()
    test = dok_matrix(train.shape) # Lưu ý hiện tại test là ma trận 0

    # với mỗi người dùng u
    # chia số trường hợp nên khuyến nghị với tỉ lệ test_ratio đươc cho
    # phần nào thuộc về test
    for u in range(n_users):
        split_index = bpr_mat[u].indices
        # đếm số trường hợp nên khuyến nghị
        count_positive = split_index.shape[0]
        n_splits = math.ceil(test_ratio * count_positive)
        test_index = np.random.choice(split_index, size=n_splits, replace=False)
        # Xem như dữ liệu chưa biết trong tập train
        train[u, test_index] = 0
        # Xem như dữ liệu nhìn thấy trong tập test
        test[u, test_index] = 1

    train, test = train.tocsr(), test.tocsr()

    # Nếu cần in thông tin ra ngoài
    if verbose:
        print('BPR matrix with %d stored elements' % bpr_mat.nnz)
        print('Train matrix with %d stored elements' % train.nnz)
        print('Test matrix with %d stored elements' % test.nnz)
    return train, test

In [8]:
bpr_train, bpr_test = split_to_train_test(bpr_mat, test_ratio=0.2, verbose=True)

BPR matrix with 82520 stored elements
Train matrix with 65641 stored elements
Test matrix with 16879 stored elements


In [9]:
def predict_bpr(W, H, user=None):
    """
    Hàm trả về X_hat

    :param W: ma trận W từ MF
    :param H: ma trận H từ MF
    :param user: người dùng (nếu None mặt định trả về tất cả)

    :return predict_scores: điểm dự đoán từ BPR MF
    """
    if user is None:
        return W @ H.T
    else:
        return W[user] @ H.T

def recommend_bpr(bpr_matrix, predict_score, user, n_rmd_items=None):
    """
    Dự đoán những sản phẩm mà người dùng muốn mua
    Những sản phẩm nào đã thích rồi thì không trả về nữa
    Trả về index theo bpr_matrix (đánh từ 0)

    :param bpr_matrix: ma trận bpr hiện tại
    :param predict_score: điểm dự đoán các item
    :param user: số thứ tự người dùng của predict score
    :param n_rmd_items: số lượng sản phẩm trả về, mặc định tất cả

    :return rmd_items: danh sách các sản phẩm khuyến nghị
    """
    # Số lượng sản phẩm
    n_items = bpr_matrix.shape[1]
    # những sản phẩm đã thích rồi
    liked_items = bpr_matrix[user].indices
    scores = predict_score.copy()

    # index ban đầu khi chưa sắp xếp
    sort_index = np.arange(0, n_items)

    # Xóa các sản phẩm đã mua
    sort_index = np.delete(sort_index, liked_items)
    scores = np.delete(scores, liked_items)

    # sắp xếp và trả về theo số thứ tự của score
    arg_sort = np.argsort(-scores)

    # dùng sort_index để lấy số thứ tự ban đầu
    rmd_items = sort_index[arg_sort]

    if len(rmd_items) >= n_rmd_items and n_rmd_items is not None:
        rmd_items = rmd_items[: n_rmd_items]
    return rmd_items

def auc_score(predict_mat, bpr_mat):
    """
    Tính Area under the ROC curve (AUC)
    cho bài toán hệ khuyến nghị

    :param predict_mat: ma trận dữ đoán bpr mf
    :param bpr_mat: ma trận train hoặc test
    :return auc: area under the roc curve
    """
    auc = 0.0
    n_users, n_items = bpr_mat.shape

    # u và row tương ứng user và bp
    for u in range(n_users):
        y_pred = predict_mat[u]
        y_true = np.zeros(n_items)
        y_true[bpr_mat[u].indices] = 1
        auc += roc_auc_score(y_true, y_pred)
    auc /= n_users
    return auc

In [10]:
def learn_bpr_mf_sgd(bpr_mat, alpha=0.01, lamb=0.01, k=12, n_iters=10000):
    """
    Thuật toán học BPR MF SGD (một điểm dữ liệu duy nhất)

    :param bpr_mat: ma trận bpr
    :param alpha: hệ số learning rate
    :param lamb: hệ số lambda của bình thường hóa regularization
    :param k: số lượng latent factor trong bài toán MF
    :param n_iters: số vòng lặp

    :return W: ma trận W
    :return H: ma trận H
    """
    n_users, n_items = bpr_mat.shape
    # Khởi tạo ma trận W và ma trận H
    W = np.ones(shape=(n_users, k))
    H = np.ones(shape=(n_items, k))
    # Tập các sản phẩm nên khuyến nghị
    pos = np.split(bpr_mat.indices, bpr_mat.indptr)[1:-1]
    # Tập các sản phẩm không nên khuyến nghị
    neg = [np.setdiff1d(np.arange(0, n_items,1), e) for e in pos]

    # lặp
    for _ in range(n_iters):
        # ngẫu nghiên 3 bộ (u,i,j) từ D_S
        u = np.random.randint(0, n_users)
        i = pos[u][np.random.randint(0, len(pos[u]))]
        j = neg[u][np.random.randint(0, len(neg[u]))]

        # Tính xuij
        xui = (W[u] * H[i]).sum()
        xuj = (W[u] * H[j]).sum()
        xuij = xui - xuj

        # mũ tự nhiên e của xuij
        exp_xuij = np.exp(xuij)

        # sgd cho tham số Theta (W và H)
        W[u] = W[u] + alpha * ( exp_xuij / (1+exp_xuij) * (H[i] - H[j]) + lamb * W[u])
        H[i] = H[i] + alpha * ( exp_xuij / (1+exp_xuij) * W[u] + lamb * H[i])
        H[j] = H[j] + alpha * ( exp_xuij / (1+exp_xuij) * (-W[u]) + lamb * H[j])
    return W, H

In [11]:
def learn_bpr_mf_mini_batch(bpr_mat, batch=100, alpha=0.01, lamb=0.01, k=12, n_iters=100):
    """
    Thuật toán học BPR MF Mini Batch

    :param bpr_mat: ma trận bpr
    :param batch: số lượng điểm dữ liệu mỗi n_iters
    :param alpha: hệ số learning rate
    :param lamb: hệ số lambda của bình thường hóa regularization
    :param k: số lượng latent factor trong bài toán MF
    :param n_iters: số vòng lặp

    :return W: ma trận W
    :return H: ma trận H
    """
    n_users, n_items = bpr_mat.shape
    # Khởi tạo ma trận W và ma trận H
    W = np.ones(shape=(n_users, k))
    H = np.ones(shape=(n_items, k))
    # Tập các sản phẩm nên khuyến nghị
    # (chuyển về numpy luôn cho tiện)
    pos = np.array(np.split(bpr_mat.indices, bpr_mat.indptr)[1:-1])
    # Tập các sản phẩm không nên khuyến nghị
    neg = np.array([np.setdiff1d(np.arange(0, n_items,1), e) for e in pos])

    # lặp
    for _ in range(n_iters):
        # mỗi u,i,j là một np array index
        u = np.random.choice(np.arange(0,n_users), batch, replace=False)
        i = []
        j = []
        for users in u:
            i.append(pos[users][np.random.randint(0, len(pos[users]))])
            j.append(neg[users][np.random.randint(0, len(neg[users]))])
        i = np.array(i)
        j = np.array(j)

        # Tính xuij
        xui = (W[u] * H[i]).sum(axis=1)
        xuj = (W[u] * H[j]).sum(axis=1)
        xuij = xui - xuj

        # mũ tự nhiên e của xuij
        exp_xuij = np.exp(xuij).reshape(batch, 1)

        # minibatch gradient descent cho Theta (W và H)
        W[u] = W[u] + alpha * ( exp_xuij / (1+exp_xuij) * (H[i] - H[j]) + lamb * W[u])
        H[i] = H[i] + alpha * ( exp_xuij / (1+exp_xuij) * W[u] + lamb * H[i])
        H[j] = H[j] + alpha * ( exp_xuij / (1+exp_xuij) * (-W[u]) + lamb * H[j])
    return W, H

In [12]:
W_sgd, H_sgd = learn_bpr_mf_sgd(bpr_train, n_iters=10000, k=12)
pred_mat_sgd = predict_bpr(W_sgd, H_sgd)

print('Train: %f' % auc_score(pred_mat_sgd, bpr_train))
print('Test: %f' % auc_score(pred_mat_sgd, bpr_test))

Train: 0.841916
Test: 0.828263


In [13]:
W_mb, H_mb = learn_bpr_mf_mini_batch(bpr_train, batch=100, n_iters=100, k=12)
pred_mat_mb = predict_bpr(W_mb, H_mb)

print('Train: %f' % auc_score(pred_mat_mb, bpr_train))
print('Test: %f' % auc_score(pred_mat_mb, bpr_test))

Train: 0.842083
Test: 0.829265


In [20]:
u = 28
n_rmd_items = 15
score = predict_bpr(W_mb, H_mb, u)
rmd_items = recommend_bpr(bpr_train, score, u, n_rmd_items)
print(rmd_items)

[257 287 180  99  49 293 312 747 332   0 327   8 236 274 120]


In [21]:
    u = 10
    n_rmd_items = 15
    score = predict_bpr(W_mb, H_mb, u)
    rmd_items = recommend_bpr(bpr_train, score, u, n_rmd_items)
    print(rmd_items)

[287 180  49 293 301 312 747 332   0 327 268 236 274 120 275]


<1x1682 sparse matrix of type '<class 'numpy.int64'>'
	with 175 stored elements in Compressed Sparse Row format>

# Tham khảo

01. Steffen Rendle, Christoph Freudenthaler, Zeno Gantner and Lars Schmidt-Thieme. BPR: Bayesian Personalized Ranking from Implicit Feedback. 
02. Weike Panand, Li Chen. GBPR: Group Preference Based Bayesian Personalized Ranking for One-Class Collaborative Filtering. Proceedings of the Twenty-Third International Joint Conference on Artificial Intelligence. https://www.aaai.org/ocs/index.php/IJCAI/IJCAI13/paper/viewFile/6316/7124
03. Michael D. Ekstrand, Joseph A Konstan. Personalized Ranking (with Daniel Kluver). Matrix Factorization and Advanced Techniques - University of Minnesota. https://www.coursera.org/lecture/matrix-factorization/personalized-ranking-with-daniel-kluver-s3XJo
04. Kim Falk. Practical Recommender Systems. Manning Publications.
05. Ethen (MingYu) Liu. Bayesian Personalized Ranking. http://ethen8181.github.io/machine-learning/recsys/4_bpr.html
06. Alfredo Láinez Rodrigo, Luke de Oliveira. Distributed Bayesian Personalized Ranking in Spark. https://stanford.edu/~rezab/classes/cme323/S16/projects_reports/rodrigo_oliveira.pdf